# **Sprint 14: Telecomunicaciones (Final project)**

El servicio de telefonía virtual CallMeMaybe está desarrollando una nueva función que brindará a los supervisores y las supervisores información sobre los operadores menos eficaces. Se considera que un operador es ineficaz si tiene una gran cantidad de llamadas entrantes perdidas (internas y externas) y un tiempo de espera prolongado para las llamadas entrantes. Además, si se supone que un operador debe realizar llamadas salientes, un número reducido de ellas también será un signo de ineficacia.

**Objetivo:** identificar operadores ineficaces; para entregarle esta información a los supervisores de manera que ellos puedan tomar desiciones informadas en las medidas que decidan tomar respecto de los operadores con más bajo rendimiento.



**Clarificaciones:** 

- Consideraré operadores ineficacies al **10% peor en cada rubro**; es decir los que estén por encima del percentil 90 o debajo del Percentil 10 según sea el caso.

- Además quiero recalcar que durante el estudio **no voy a considerar a los trabajadores ineficaces como un grupo homogeneo**, sino que voy a encontrar cuales de los operadores están fallando en cada uno de los rubros, y los compararé con el resto del grupo.

- Sin embargo al final tendré 3 listas de ID's de operadores ineficientes una por cada rubro, las cuales uniré y eliminaré duplicados para tener **una única lista de operadores ineficaces**. 

Esto lo estoy haciendo porque al tratarlos por separado no sesgo los datos, soy más preciso al hacer las comparaciones, contrastando rubro por rubro. Además si los supervisores lo desean podrían revisar en qué están fallando los operadores, quizá no quieran simplemente despedirlos, sino ayudarlos a mejorar sus métricas. También es interesante ver cuales operadores aparecen en 2 o en las 3 listas; al final puedo tener una pivot table con los ID de los operadores ineficientes que indique en cuales rubros falla cada uno. Por todo lo anterior es que considero relevante tratarlo de esta manera.

## **Hipótesis:** 

1. "Los operadores considerados ineficaces tienen una tasa de llamadas perdidas (internas y externas) significativamente mayor que la media de los operadores eficaces."

2. "Los operadores ineficaces tienen un tiempo promedio de espera en llamadas entrantes significativamente mayor que el de los operadores eficaces."

3. "Los operadores ineficaces realizan significativamente menos llamadas salientes que la media de los operadores eficientes"

## **Plan de acción** 

**1. Preprocesamiento de datos**
- Estudiar valores ausentes
- Estudiar el tipo de correspondencia
- Estudiar valores duplicados

**2. Análisis exploratorio de datos**

**3. Agrupar los datos por operator_id**

**4. Métodos para cada rubro:**
- Por la cantidad de llamadas perdidas (.sum())
- Por los tiempos de espera en llamadas entrantes (.mean())
- Por la cantidad de llamadas salientes (.sum())

**5. Encontrar los percentiles respectivos de cada rubro**
- P90 (Cuando mucho es ineficiente)
- P10 (Cuando poco es ineficiente)

**6. Separar a los operadores ineficientes y los eficientes de cada rubro**

**7. Comparar estadísticamente con una prueba T los grupos de cada rubro**

**8. Crear una pivot table con los usuarios ineficientes**

**9. Conclusiones**

# **Step 1: Data preprocessing**

## **Import libraries**

In [3]:
#!pip install pandas

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import ttest_ind
import math as mth
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## **Read the data**